# R Aqr offsets calculations
Calculate x, y offsets for R Aqr target acquisition.

In [1]:
# Set up autoloader
%load_ext autoreload
%autoreload 2

In [30]:
# Set up modules and stuff
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, Distance

## Date range for JWST observation
This is the date range for the scheduled JWST observation

In [47]:
# JWST dates and times
JWST_TIME1 = '2024-10-30T18:19:00.000'    # Beginning of JWST observing window
JWST_TIME2 = '2024-12-12T00:19:55.000'    # Beginning of JWST observing window
JWST_TIME = '2024-11-06T18:19:00.000'     # Schediuled time JWST time

# Create Time-like objects with the JWST times
jwst_time1 = Time(JWST_TIME1, format='isot', scale='utc')
jwst_time2 = Time(JWST_TIME2, format='isot', scale='utc')
jwst_time = Time(JWST_TIME, format='isot', scale='utc')

## Reference coordinates for R Aqr and offset star
This notebook provides two sets of coordinates for R Aqr (since the target had been observed twice), and the Gaia DR3 Coodinates of the offset star. The most recent set is the one actually used to calculate the updated coordinates at the time of the JWST observation. The earlier set of coordinates is used to test that the routine work properly (by using them as reference coordinates and checking that the result matches).

In [46]:
# R Aqr coordinates set 1
RA = '23:43:49.50120 hours'     # RA in sexagesimal format (ICRS)
DEC = '-15:17:04.804 degrees'   # DEC in sexagesimal format (ICRS) 
PMRA = 27.33                    # Proper motion in RA (mas/yr)
PMDEC = -30.37                  # Proper motion in DEC (mas/yr)
PARALLAX = 2.5931               # Parallax (mas)
OBS_DATE = '2020-10-01'         # Observation date
EPOCH = 2000.0                  # Reference epoch
FRAME = 'icrs'                  # Coordinate frame

# Create Skycoord object with the given coordinates
r_aqr1 = SkyCoord(RA,
                  DEC,
                  pm_ra_cosdec=PMRA*u.mas/u.yr,
                  pm_dec=PMDEC*u.mas/u.yr,
                  distance=Distance(parallax=PARALLAX*u.mas),
                  equinox=Time(EPOCH, format='jyear'),
                  obstime=Time(OBS_DATE, format='iso', scale='utc'),
                  frame=FRAME
                 )
print(r_aqr1)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (355.956255, -15.28466778, 385.63881069)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (27.33, -30.37)>


In [45]:
# R Aqr coordinates set 2
RA = '23:43:49.50785 hours'     # RA in sexagesimal format (ICRS)
DEC = '-15:17:04.911 degrees'   # DEC in sexagesimal format (ICRS) 
PMRA = 27.33                    # Proper motion in RA (mas/yr)
PMDEC = -30.37                  # Proper motion in DEC (mas/yr)
PARALLAX = 2.5931               # Parallax (mas)
OBS_DATE = '2024-04-07'         # Observation date
EPOCH = 2000.0                  # Reference epoch
FRAME = 'icrs'                  # Coordinate frame

# Create Skycoord object with the given coordinates
r_aqr2 = SkyCoord(RA,
                  DEC,
                  pm_ra_cosdec=PMRA*u.mas/u.yr,
                  pm_dec=PMDEC*u.mas/u.yr,
                  distance=Distance(parallax=PARALLAX*u.mas),
                  equinox=Time(EPOCH, format='jyear'),
                  obstime=Time(OBS_DATE, format='iso', scale='utc'),
                  frame=FRAME
                 )
print(r_aqr2)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (355.95628271, -15.2846975, 385.63881069)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (27.33, -30.37)>


In [44]:
# Gaia DR3 2419576423272148480 coordinates
RA = '355.94560363551375 degrees'    # RA in decimal degrees (ICRS)
DEC = '-15.274577751934695 degrees'  # DEC in decimal degrees (ICRS) 
PMRA = -2.708690975158548            # Proper motion in RA (mas/yr)
PMDEC = -4.0809858570655875          # Proper motion in DEC (mas/yr)
OBS_DATE = 2016.0                    # Observation date
EPOCH = 2016.0                       # Reference epoch
FRAME = 'icrs'                       # Coordinate frame

# Create Skycoord object with the given coordinates
acqt = SkyCoord(RA,
                DEC,
                pm_ra_cosdec=PMRA*u.mas/u.yr,
                pm_dec=PMDEC*u.mas/u.yr,
                equinox=Time(EPOCH, format='jyear'),
                obstime=Time(OBS_DATE, format='jyear', scale='tcb'),
                frame=FRAME
            )
print(acqt)

<SkyCoord (ICRS): (ra, dec) in deg
    (355.94560364, -15.27457775)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (-2.70869098, -4.08098586)>
